[View in Colaboratory](https://colab.research.google.com/github/tjwei/tensorflow-tutorial/blob/master/gan/AutoEncoder.ipynb)

In [0]:
import keras
from PIL import Image
import numpy as np

In [0]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [0]:
train_X.shape

In [0]:
train_X.dtype

In [0]:
Image.fromarray(train_X[1])

In [0]:
train_X = (train_X[..., None]-127.5)/128

In [0]:
train_X.shape

In [0]:
from IPython.display import display
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:10])
print(train_y)

In [0]:
NZ = 8

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D

netE = Sequential([
    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=(28,28,1)),
    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
    Conv2D(filters=NZ, kernel_size=3, strides=2, padding='valid'),
    GlobalAveragePooling2D(),
    Activation('tanh')
    
])

In [0]:
netE.summary()

In [0]:
from keras.layers import Conv2DTranspose, Reshape
netG = Sequential([
    Reshape( (1,1,NZ), input_shape=(NZ,)),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='valid', activation='relu'),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='valid', activation='relu'),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
    Conv2DTranspose(filters=1, kernel_size=3, strides=2, padding='same')
    
])

In [0]:
netG.summary()

In [0]:
from keras.models import Model
EG_output = netG(netE.outputs)
netEG = Model(inputs=netE.inputs, outputs=[EG_output])

In [0]:
netEG.summary()

In [0]:
import numpy as np
showX(netG.predict(np.random.normal( size=(10, NZ))))

In [0]:
netEG.compile(loss='mse', optimizer='adam')

In [0]:
netEG.fit(train_X, train_X, epochs=1)

In [0]:
showX(netG.predict(np.random.normal(size=(100, NZ))), 10)

In [0]:
showX(netEG.predict(train_X[:100]), 10)

In [0]:
codes = netE.predict(train_X[:1000])
#codes = netE.predict(train_X[train_y==2])

In [0]:
codes.mean(axis=0)

In [0]:
codes.std(axis=0)

In [0]:
z = np.random.normal(codes.mean(axis=0), scale=codes.std(axis=0), size=(100,NZ))
showX(netG.predict(z), 10)
